In [ ]:
#Uncomment these if running in colab
!pip install wget
!pip install newspaper3k
!pip install xmltodict
!pip install pandarallel
!pip install datefinder
!pip install pydrive
!pip install selenium
!pip3 install "urllib3 <=1.26.15"
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# <font color='red'>If imports give you any errors, uncomment the install cell above.</font>
### You only need to install these libraries once.
### Future runs or scripts, you can comment them out to save time.

In [ ]:
import sys
import wget
import newspaper
from newspaper import Article
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
import lxml
import xmltodict
import collections
from urllib import request
from collections import OrderedDict
from urllib.error import HTTPError, URLError
from urllib.parse import urlparse
import sys
import ast
import time
from pandarallel import pandarallel
import requests
import datefinder
from datetime import datetime
from tqdm import tqdm
import psutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from multiprocessing import Pool
import random
from urllib.parse import quote
import pytz




# <font color='red'>The next cell you need to update for each site you scrape </font>
# <font color='red'>I recommend creating a sperate script for each site you scrape. </font>
# <font color='red'>Test scraping of few pages, to check for errors, before scraping all site </font>


### > Name the script above from HPC NEW Scraper to the site name.

### > Please remember to fill/update news outlet and country for site

### > Remember to change "types" depending on page if it's numbered by pages, or click more or scroll down [0,1,2]
### > max_click_SHOW_MORE = the number of clicks or scroll down you like.


### > extract_text_sleep = sleep amount for extracting text
### > extract_urls_sleep = sleep amount for extracting links

### > Remmember to update page identifer on line 39 if website is a pages site






In [ ]:
t0 = time.time()

links_file_path = ""
csv_file_path = ""

#PLEASE EDIT THIS CELL!

#Universal parameters. Always needs to be changed
news_outlet = ""
country = ""
max_click_SHOW_MORE = 500
host = ""
date_format = "%Y-%m-%d"
relevant_path = f'^{host}.*$'
# Allow http and https links
if re.match("^https?.*", host):
  relevant_path = f'^https?:\/\/{host[8:]}.*$'
else:
  relevant_path = f'^https?:\/\/{host}.*$'
urls = ['/'] #if section url has arabic use "quote(/news/سياسة)"
urls = [host + url for url in urls]
csv_name = country+'_'+news_outlet+'.csv'
types = ['page', 'click_more','scroll_down']
type_of_page = types[0]
extract_text_sleep = random.randint(10,12)
extract_urls_sleep = random.randint(10,12)


# If page type is click_more or scroll_down i.e. can get more article by clicking a button with text 'click more' or simply scrolling down
#-----------------------------------------------------------------------------------------------------------------------------------------
xpath_for_link = ""
xpath_for_click_more = ""
#-----------------------------------------------------------------------------------------------------------------------------------------


# If page type is page
#-----------------------------------------------------------------------------------------------------------------------------------------
target_tag = ''
target_tag_class = ""

page_identifier = ''

if type_of_page == 'page':
    urls = [url + page_identifier for url in urls]
pages_each_category = [1] #The total number of pages for each section. So if policiaca is 1..92, write 92, not 93.
if pages_each_category:
    total_pages = pages_each_category
else:
    total_pages = 500
#-----------------------------------------------------------------------------------------------------------------------------------------

#Get amount of pages per category not having text code
#-----------------------------------------------------------------------------------------------------------------------------------------
retrieve_amount_of_pages_no_text = False
#-----------------------------------------------------------------------------------------------------------------------------------------


#Get amount of pages per category with text code
#-----------------------------------------------------------------------------------------------------------------------------------------
retrieve_amount_of_pages_w_text = False
xpath_for_amount_of_pages = ""
index_for_amount_of_pages_location = 0
#-----------------------------------------------------------------------------------------------------------------------------------------

# in case multi tags
target_tags = []
target_tag_classes = []



In [ ]:
def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
# Functions for automatically identifying the number of pages

def check_it_has_links(url):
  options = FirefoxOptions()
  options.add_argument("--headless")
  driver = webdriver.Firefox(options=options)
  driver.get(url)

  has_links= True
  try:
    element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, xpath_for_link))
    )
  except TimeoutException as e:
    print('Could not find pages count')
    has_links = False
  driver.quit()
  return has_links

def retrieve_pages_per_category_no_text(url):
  pages = 1
  pages_defined = False

  while check_it_has_links(url + f'{pages}'):
    pages = pages * 10

  add_pages = pages // 2
  if pages != 1:
    pages = pages // 2
    while not pages_defined:
      add_pages = add_pages // 2
      if check_it_has_links(url + f'{pages}'):
        pages = add_pages + pages
      else:
        pages = pages - add_pages
      if add_pages == 1:
        pages_defined = True
      print(pages)

  return pages

def retrieve_pages_per_category_w_text(url):
  options = FirefoxOptions()
  options.add_argument("--headless")
  driver = webdriver.Firefox(options=options)
  driver.get(url)
  try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath_for_amount_of_pages))
    )
  except TimeoutException as e:
    print('Could not find pages count')
    return ''
  pages = driver.find_element(By.XPATH, xpath_for_amount_of_pages).text
  if not pages:
    print('No text found')
  driver.quit()
  return pages

In [ ]:
if retrieve_amount_of_pages_no_text:
  total_pages = []
  for url in urls:
    amount_of_pages = retrieve_pages_per_category_no_text(url)
    total_pages.append(amount_of_pages)
  print(total_pages)

if retrieve_amount_of_pages_w_text:
  total_pages = []
  for url in urls:
    text_with_amount_of_pages = retrieve_pages_per_category_w_text(url)

    amount_of_pages =[]

    for char in text_with_amount_of_pages[index_for_amount_of_pages_location:]:
      if not char.isnumeric():
        if char != ',' and char != '.':
          break
      else:
        amount_of_pages.append(char)
    amount_of_pages = ''.join(amount_of_pages)
    print(amount_of_pages + ' ' + url)
    if amount_of_pages:
      total_pages.append(int(amount_of_pages))
    else:
      total_pages.append(1)
  print(total_pages)

In [ ]:

output_path = 'links_'+csv_name

def retrieve_links_from_list(url,scroll_down=False):
  options = FirefoxOptions()
  options.add_argument("--headless")
  driver = webdriver.Firefox(options=options)
  driver.get(url)

  count = 0

  news_urls = set()

  prev_length = 0
  while count <= max_click_SHOW_MORE:
      if not scroll_down:
          try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, xpath_for_click_more))
            )
          except TimeoutException as e:
            print('Could not find click more button')
            break


      try:
          all_links = driver.find_elements(By.XPATH, xpath_for_link)
          if not scroll_down:
            click_more = driver.find_element(By.XPATH, xpath_for_click_more)
          curr_length = len(all_links)
          print(f'{curr_length} links in current page')
          print(f'{prev_length} links in previous page')
          if count > 0:
            if curr_length == prev_length or (not scroll_down and not click_more):
              break

          current_links = [l.get_attribute("href") for l in all_links[prev_length:]]
          df_link = pd.DataFrame(current_links, columns = ['link'])
          df_link.to_csv(output_path, mode='a', header=not os.path.exists(output_path))

          prev_length = curr_length

          if not scroll_down:

            try:
                click_more.click();
            except Exception as e:
                driver.execute_script("arguments[0].click();", click_more) #If click does not work because of overlapping elements, this executes

            print(f"Button clicked {count} times", )
          else:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            print(f"Scrolled down {count+1} times")

          time.sleep(3)

          count += 1



      except TimeoutException:
          break

  time.sleep(2)

  all_links = driver.find_elements(By.XPATH, xpath_for_link)

  for link in all_links:
        if isinstance(link.get_attribute('href'), str):
          if re.match(relevant_path, link.get_attribute('href')):
            news_urls.add(link.get_attribute('href'))

  driver.quit()
  print('*' * 20)
  return list(news_urls)


def extract_urls(url):
    rows = set()
    soup = getSoup(url)
    if soup:
      all_elems = None
      if len(target_tags) > 0 and len(target_tag_classes) > 0:
        all_elems = []
        for tag in target_tags:
          for tag_class in target_tag_classes:
            all_elems += soup.find_all(tag, {'class':tag_class})
      elif target_tag_class != '':
        all_elems = soup.find_all(target_tag, {'class':target_tag_class})
      else:
        all_elems = soup.find_all(target_tag)

      for d in all_elems:
        all_links = d.find_all('a', href=True)
        if not all_links:
            all_links = [d]
        for l in all_links:
            if not re.match('^http.*', l['href']):
              l['href'] = host + l['href']

            if re.match(relevant_path, l['href']):
              rows.add(l['href'])


    if rows:
      return list(rows)
    else:
      return None

def extract_urls_from_df(df):
    links = df['page'].map(lambda x: extract_urls(x))
    df['link'] = links
    return df

In [ ]:
pages = []

def get_zipped_urls(url, total_pages):
    page_numbers = [str(i) for i in range(1,total_pages)]
    url_multiple = [url] * total_pages
    return [''.join(x) for x in zip(url_multiple,page_numbers)]

if type_of_page == 'page':
    for ind, url in enumerate(urls):
        if isinstance(total_pages, int):
            pages += get_zipped_urls(url, total_pages)
        else:
            pages_for_category = total_pages[ind] + 1
            pages += get_zipped_urls(url, pages_for_category)

elif type_of_page == 'click_more' or type_of_page == 'scroll_down':
    for url in urls:
        pages += retrieve_links_from_list(url, True if type_of_page == 'scroll_down' else False)


df = pd.DataFrame(pages, columns = ['page'])
df

In [ ]:
def getSoup(url):
    """
    Return a soup object of the URL
    """
    try:
        req = request.Request(url, headers={'User-Agent' : "Chrome"})
        con = request.urlopen(req)
        time.sleep(extract_urls_sleep)
        html = con.read()

    except HTTPError as e:
        print(e)
        return None

    except URLError as e:
        print('The server could not be found')
        return None

    except Exception as e:
      print(e)
      return None


    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [ ]:
#New Code caused issues

def extract_text(url):
    text, title, date = None, None, None
    try:
        result = [country, news_outlet] + [''] * 3
        article = Article(url, keep_article_html=False)
        article.download()
        time.sleep(extract_text_sleep)
        article.parse()
        text = article.text

        if text:
            text_copy = text
            title = article.title
            date = article.publish_date
            if date:
                date = date.strftime(date_format)
            else:
                matches = datefinder.find_dates(text_copy)
                most_recent_datetime = sorted(matches)[-1]
                date = most_recent_datetime.strftime(date_format)
    finally:
      if title:
        result[2] = title
      if date:
        result[3] = date
      if text:
        result[4] = text
      return result


def extract_text_from_df(df):
    content = df['link'].map(lambda x: extract_text(x))
    df['content'] = content
    return df

In [ ]:
df

In [ ]:
def parallel_work(df, method_to_run, target, source):
  global workers
  while workers>0:
    try:
      df[target] = df[source].parallel_apply(method_to_run)
    except Exception as e:
      raise e
    break

  if workers == 0:
    print('Error during parallel operation. Could not extract text')

  return df

def get_parallel_operation_results(divided_dfs, method_to_run, target, source):
  res = []
  for df in divided_dfs:
    try:
      temp_df = parallel_work(df, method_to_run, target, source)
      if temp_df[target].isnull().all():
        print('Could not retrieve any URLS')
        print('Something is wrong with the target_tag and target_tag_class variables. Please modify')
        return []
      res.append(temp_df)
    except Exception:
      continue
  df_result = pd.concat(res)
  return df_result

def partition_df(df):
  global articles_per_parallel_operation
  divided_dfs = []
  start = 0
  while start < len(df):
    divided_dfs.append(df[start:start+articles_per_parallel_operation])
    start += articles_per_parallel_operation
  return divided_dfs

In [ ]:
if type_of_page == 'page':
    start = 0
    limit = 40
    total_time_start = time.time()
    results = []

    while start < len(df):
        start_time = time.time()
        results.append(parallelize_dataframe(df[start:start+limit], extract_urls_from_df, 94)) ## NUMBER of CORES here
        end_time = time.time()
        print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
        print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
        start+=limit

    df = pd.concat(results)
    total_time_end = time.time()
    print(f'total time taken: {round(total_time_end - total_time_start,2)} second')

else:
    df = df.rename(columns={"page": "link"})


# <font color='red'> If you're having problems extracting links (HTTP 500, 503, etc) </font>

### > Double check in browser the links from 'page' column below to see if they're correct.
### > If they're not working, correct your host, url and page identifier you filled in above.

## <font color='red'> If links are correct </font>
### > Double check your html tags that reference article urls.
### > Try other tags that reference the article link, such as links embedded in article picture or a separate read more button that leads to article page

## <font color='red'> If tags correct </font>
### > Increase the sleep time of extract_urls_sleep in set up cell.
### > So if it was random.randint(5,7), set it to random.randint(10,12)


## <font color='red'> if issue still persist </font>
### > Try to reduce number of cores in above cell from 94 to a lower number
### > For example, if you're getting HTTP 500 errors with 94 cores, I will lower it down to 50. If that doesn't work 25 and so on!

# <font color='red'> If all that doesn't work and site is blocking you, just move on to next site and list reason/issue in excel file :) </font>

In [ ]:
df

In [ ]:
df.to_csv(links_file_path+csv_name)

In [ ]:
print(len(df))
df = df[df['link'].notna()]
df = df[(df['link'].str.len() != 0)]
print(len(df))

In [ ]:
if type_of_page == 'page':
    df = df.explode('link')
df

In [ ]:
print(len(df))
df = df.drop_duplicates(subset='link')
print(len(df))

# <font color='red'> IMPORTANT - UPDATE LINE 4 NEXT CELL WITH YOUR "links" SAVE DIRECTORY </font>

## Update this Path below to reflect your special workspace '/xdisk/josorio1/conflibert_esp/Summer_2023/Sultan/links/

## >> For SPANISH <<
### Go to https://ood.hpc.arizona.edu/pun/sys/dashboard/files/fs//xdisk/josorio1/conflibert_esp/Summer_2023
### Find your directory and create folder named 'links' inside of it.
### Go inside 'links' folder and click copy path and paste it below.
### Make sure you close your copied path with "/"

## >> For Arabic <<
### Go to https://ood.hpc.arizona.edu/pun/sys/dashboard/files/fs//xdisk/josorio1/conflibert_ara/Summer_2023
### Find your directory and create folder named 'links' inside of it.
### Go inside 'links' folder and click copy path and paste it below.
### Make sure you close your copied path with "/"

In [ ]:
#PLEASE UPDATE
#Create a links checkpoint csv
df.to_csv('links_'+csv_name)
df.to_csv(links_file_path+'links_'+csv_name, encoding = 'utf-8-sig')

In [ ]:
if 'df' not in locals() or df is None or 'link' not in df:
    if(os.path.exists('links_'+csv_name)):
        df = pd.read_csv('links_'+csv_name, index_col=0)

In [ ]:
df

In [ ]:
output_path= 'unprocessed_' + csv_name
start = 0
limit = 40

res = []
while start < len(df):
    start_time = time.time()
    demo_df = df[start:start+limit].copy()
    test_df = parallelize_dataframe(demo_df, extract_text_from_df, 94) #NUMBER OF CORES HERE
    res.append(test_df)
    test_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))

    test_df['text_set'] = test_df['content'].map(lambda x: True if x[4] != '' else False)
    no_of_text_retrieved = test_df.text_set.sum()
    print(f'{no_of_text_retrieved} / {len(test_df)} text retrieved')

    end_time = time.time()
    print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
    print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
    start+=limit

df = pd.concat(res)

In [ ]:
output_path= 'unprocessed_' + csv_name
if 'df' not in locals() or df is None or 'content' not in df:
    if(os.path.exists(output_path)):
#         df = pd.read_csv(output_path, on_bad_lines='skip', converters={'content': pd.eval}, index_col=0)
        df = pd.read_csv(output_path, on_bad_lines='skip', index_col=0)

# <font color='red'> If you're having problems extracting links (0 / 40 text retrieved) </font>

### Double check in you browser the articles links you extracted above in 'link' column to see if they're working.
### If they're not, correct your host you filled in above. You might have an extra '/' or repeated text in links such as '/es/es/news'

## <font color='red'> If links are correct </font>
### > Increase the sleep time of extract_urls_sleep in set up cell.
### > So if it was random.randint(5,7), set it to random.randint(10,12)


## <font color='red'> if issue still persist </font>
### > Try to reduce number of cores in above cell from 94 to a lower number
### For example, if you're getting 0 / 40 errors with 94 cores, lower it down to 50. If that doesn't work 25 and so on!

# <font color='red'> If all that doesn't work and site is blocking you, just move on to next site and list reason/issue in excel file :) </font>


In [ ]:
df[['country', 'news_outlet', 'title', 'date', 'text']] = pd.DataFrame(df.content.tolist(), index= df.index)
df


In [ ]:
print(len(df))
df = df[df['text'] != '']
print(len(df))
df

In [ ]:
if 'page' in df:
  df = df.drop(['page'], axis=1)
if 'content' in df:
  df = df.drop(['content'], axis=1)
if 'Unnamed: 0' in df:
    df = df.drop(['Unnamed: 0'], axis=1)
if 'Unnamed: 0.1' in df:
    df = df.drop(['Unnamed: 0.1'], axis=1)

In [ ]:
df

# <font color='red'> IMPORTANT - UPDATE LINE 5 NEXT CELL WITH YOUR "csv" SAVE DIRECTORY </font>

## Update this Path below to reflect your special workspace '/xdisk/josorio1/conflibert_esp/Summer_2023/Sultan/csv/'

## >> For SPANISH <<
### Go to https://ood.hpc.arizona.edu/pun/sys/dashboard/files/fs//xdisk/josorio1/conflibert_esp/Summer_2023
### Find your directory and create folder named 'csv' inside of it.
### Go inside 'csv' folder and click copy path and paste it below.
### Make sure you close your copied path with "/"

## >> For Arabic <<
### Go to https://ood.hpc.arizona.edu/pun/sys/dashboard/files/fs//xdisk/josorio1/conflibert_ara/Summer_2023
### Find your directory and create folder named 'csv' inside of it.
### Go inside 'csv' folder and click copy path and paste it below.
### Make sure you close your copied path with "/"

In [ ]:
#Please update
seconds = time.time() - t0
duration = time.strftime("(%H:%M:%S)",time.gmtime(seconds))

df.to_csv(csv_file_path+duration+'-'+csv_name, encoding = 'utf-8-sig')

print('Time to complete:',duration)

#tz_DFW = pytz.timezone('US/Central')
#current_time = datetime.now(tz_DFW)
#time = current_time.strftime("(%H:%M:%S)")



#df.to_csv(time+csv_name)

In [ ]:
if(os.path.exists(output_path)):
    os.remove(output_path)
    print(('unprocessed file deleted'))